In [31]:
import torch
from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset("csv",data_dir="./", data_files="data_cleaned_5000.csv", split=split)
        # self.dataset = load_dataset("csv",
        # data_dir="C:/Users/19148/Documents/Pycharm_projects/paper_project", data_files="data_lite.csv", split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label


dataset = Dataset('train')

len(dataset), dataset[15]

Using custom data configuration default


Dataset csv downloaded and prepared to C:\Users\19148\.cache\huggingface\datasets\csv\default-4177d6418d7b1aec\0.0.0\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


C:\Users\19148\.cache\huggingface\modules\datasets_modules\datasets\csv\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2\csv.py:91: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_file_reader = pd.read_csv(
C:\Users\19148\.cache\huggingface\modules\datasets_modules\datasets\csv\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2\csv.py:91: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  csv_file_reader = pd.read_csv(


(5596, ('求解答这里是讲收敛数列的有界性，为什么举例是这个例子这个数列并不收敛呀这个数列不是发散的吗', 0))

In [13]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [32]:
batch_size = 16
max_len = 300

In [33]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=max_len,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=batch_size,
                                     collate_fn=collate_fn,
                                     # shuffle=False,
                                     shuffle=True,
                                     drop_last=True)
len_loader = len(loader)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    print("%d/%d"%(i+1,len_loader))
    # print(labels)
    # print(input_ids.shape)
    # break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

1/349
2/349
3/349
4/349
5/349
6/349
7/349
8/349
9/349
10/349
11/349
12/349
13/349
14/349
15/349
16/349
17/349
18/349
19/349
20/349
21/349
22/349
23/349
24/349
25/349
26/349
27/349
28/349
29/349
30/349
31/349
32/349
33/349
34/349
35/349
36/349
37/349
38/349
39/349
40/349
41/349
42/349
43/349
44/349
45/349
46/349
47/349
48/349
49/349
50/349
51/349
52/349
53/349
54/349
55/349
56/349
57/349
58/349
59/349
60/349
61/349
62/349
63/349
64/349
65/349
66/349
67/349
68/349
69/349
70/349
71/349
72/349
73/349
74/349
75/349
76/349
77/349
78/349
79/349
80/349
81/349
82/349
83/349
84/349
85/349
86/349
87/349
88/349
89/349
90/349
91/349
92/349
93/349
94/349
95/349
96/349
97/349
98/349
99/349
100/349
101/349
102/349
103/349
104/349
105/349
106/349
107/349
108/349
109/349
110/349
111/349
112/349
113/349
114/349
115/349
116/349
117/349
118/349
119/349
120/349
121/349
122/349
123/349
124/349
125/349
126/349
127/349
128/349
129/349
130/349
131/349
132/349
133/349
134/349
135/349
136/349
137/349
138/349
139/

(torch.Size([16, 300]),
 torch.Size([16, 300]),
 torch.Size([16, 300]),
 tensor([3, 1, 0, 3, 0, 3, 1, 1, 3, 2, 0, 1, 0, 2, 1, 0]))

In [34]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

print(out.last_hidden_state.shape)
# print(out.last_hidden_state[1,0])
w2v = torch.zeros(len(dataset), 768)
label = torch.zeros(len(dataset))
len_loader = len(loader)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader): # 这样就行了
    print("%d/%d"%(i+1,len_loader),end=' ')
    if i%8 == 7:
        print('\n', end='')
    out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)
    vec = out.last_hidden_state
    for j in range(batch_size):  # batch_size=16
        w2v[batch_size * i + j] = torch.mean(vec[j], dim=0, keepdim=True)  # 取均值，而不是第一个值
        # w2v[batch_size * i + j] = vec[j,0]  # vector
        label[batch_size * i + j] = labels[j]
print(w2v.shape, label.shape)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 300, 768])
1/349 2/349 3/349 4/349 5/349 6/349 7/349 8/349 
9/349 10/349 11/349 12/349 13/349 14/349 15/349 16/349 
17/349 18/349 19/349 20/349 21/349 22/349 23/349 24/349 
25/349 26/349 27/349 28/349 29/349 30/349 31/349 32/349 
33/349 34/349 35/349 36/349 37/349 38/349 39/349 40/349 
41/349 42/349 43/349 44/349 45/349 46/349 47/349 48/349 
49/349 50/349 51/349 52/349 53/349 54/349 55/349 56/349 
57/349 58/349 59/349 60/349 61/349 62/349 63/349 64/349 
65/349 66/349 67/349 68/349 69/349 70/349 71/349 72/349 
73/349 74/349 75/349 76/349 77/349 78/349 79/349 80/349 
81/349 82/349 83/349 84/349 85/349 86/349 87/349 88/349 
89/349 90/349 91/349 92/349 93/349 94/349 95/349 96/349 
97/349 98/349 99/349 100/349 101/349 102/349 103/349 104/349 
105/349 106/349 107/349 108/349 109/349 110/349 111/349 112/349 
113/349 114/349 115/349 116/349 117/349 118/349 119/349 120/349 
121/349 122/349 123/349 124/349 125/349 126/349 127/349 128/349 
129/349 130/349 131/349 132/349 133/349 1

In [35]:
import numpy as np
import pandas as pd
w2v_list = w2v.tolist()
# l_list = label.tolist()
total = np.concatenate((w2v_list, label.reshape(len(label), 1)), axis=1)
tp = pd.DataFrame(total)
# tp.to_csv("w2v.csv", header=None, index=None)
tp.to_csv("w2v_cleaned_mean_shuffle_5000.csv", header=None, index=None)

In [25]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 300)
    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
        out = self.fc(out.last_hidden_state)
        # out = out.softmax(dim=1)
        return out
model = Model()
w2v_low_dim = torch.zeros(len(dataset), 300)
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader): # 这样就行了
    print("%d/%d"%(i+1,len_loader),end=' ')
    if i%8 == 7:
        print('\n', end='')
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
    for j in range(batch_size):  # batch_size=16
        # w2v_low_dim[batch_size * i + j] = out[j,0]  # vector
        # 取均值，而不是第一个值
        w2v_low_dim[batch_size * i + j] = torch.mean(out[j], dim=0, keepdim=True)
print(w2v_low_dim.shape)

1/229 2/229 3/229 4/229 5/229 6/229 7/229 8/229 
9/229 10/229 11/229 12/229 13/229 14/229 15/229 16/229 
17/229 18/229 19/229 20/229 21/229 22/229 23/229 24/229 
25/229 26/229 27/229 28/229 29/229 30/229 31/229 32/229 
33/229 34/229 35/229 36/229 37/229 38/229 39/229 40/229 
41/229 42/229 43/229 44/229 45/229 46/229 47/229 48/229 
49/229 50/229 51/229 52/229 53/229 54/229 55/229 56/229 
57/229 58/229 59/229 60/229 61/229 62/229 63/229 64/229 
65/229 66/229 67/229 68/229 69/229 70/229 71/229 72/229 
73/229 74/229 75/229 76/229 77/229 78/229 79/229 80/229 
81/229 82/229 83/229 84/229 85/229 86/229 87/229 88/229 
89/229 90/229 91/229 92/229 93/229 94/229 95/229 96/229 
97/229 98/229 99/229 100/229 101/229 102/229 103/229 104/229 
105/229 106/229 107/229 108/229 109/229 110/229 111/229 112/229 
113/229 114/229 115/229 116/229 117/229 118/229 119/229 120/229 
121/229 122/229 123/229 124/229 125/229 126/229 127/229 128/229 
129/229 130/229 131/229 132/229 133/229 134/229 135/229 136/229 
137

In [17]:
print(w2v_low_dim.shape)
fc = torch.nn.Linear(768, 300)
temp = fc(w2v)
print(temp.shape)

torch.Size([3664, 300])
torch.Size([3664, 300])


In [26]:
import numpy as np
import pandas as pd
w2v_low_dim_list = w2v_low_dim.tolist()
# w2v_low_dim_list = temp.tolist()
total_low_dim = np.concatenate((w2v_low_dim_list, label.reshape(len(label), 1)), axis=1)
tq = pd.DataFrame(total_low_dim)
# tp.to_csv("w2v.csv", header=None, index=None)
tq.to_csv("w2v_big_low_dim_mean.csv", header=None, index=None)

In [23]:
#定义下游任务模型
# class Model(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         # self.fc = torch.nn.Linear(768, 2)
#
#     def forward(self, input_ids, attention_mask, token_type_ids):
#         with torch.no_grad():
#             out = pretrained(input_ids=input_ids,
#                        attention_mask=attention_mask,
#                        token_type_ids=token_type_ids)
#
#         out = self.fc(out.last_hidden_state)
#
#         # out = out.softmax(dim=1)
#
#         return out
#
#
# model = Model()
#
# out = model(input_ids=input_ids,
#       attention_mask=attention_mask,
#       token_type_ids=token_type_ids)
# print(out)
# out.shape

In [6]:
from transformers import AdamW

#训练
# optimizer = AdamW(model.parameters(), lr=5e-4)
# criterion = torch.nn.CrossEntropyLoss()

# model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader): # 这样就行了
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

0 0.6782209873199463 0.625
5 0.5840441584587097 0.75
10 0.6991336941719055 0.375
15 0.6459898948669434 0.625
20 0.6601291298866272 0.5625
25 0.6269813776016235 0.8125
30 0.595515251159668 0.875
35 0.5542500019073486 0.875
40 0.5360901355743408 0.8125
45 0.5258952379226685 0.9375
50 0.5396168828010559 0.875
55 0.5625669956207275 0.75
60 0.503290057182312 0.875
65 0.5144746899604797 0.75
70 0.4966588318347931 0.875
75 0.5100283622741699 0.8125
80 0.5462980270385742 0.6875
85 0.5015004873275757 0.9375
90 0.4921759068965912 0.875
95 0.5301690697669983 0.8125
100 0.4322471618652344 0.9375
105 0.4398854672908783 1.0
110 0.6205551028251648 0.6875
115 0.4555570185184479 0.9375
120 0.43458104133605957 0.9375
125 0.5856747031211853 0.8125
130 0.45579853653907776 0.875
135 0.49450209736824036 0.875
140 0.4834059476852417 0.875
145 0.41298091411590576 0.9375
150 0.6239964365959167 0.6875
155 0.42134153842926025 0.9375
160 0.41760149598121643 1.0
165 0.4275535047054291 0.9375
170 0.5800575613975525

In [7]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

Using custom data configuration default
Reusing dataset chn_senti_corp (/Users/lee/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


0
1
2
3
4
0.86875
